<a href="https://colab.research.google.com/github/jayeshvpatil/llm_colabs/blob/main/Fine_Tuning_LLM_Med_Q%26A_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install accelerate peft  transformers  datasets evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 16407
    })
})

In [6]:
dataset['train'][0]

{'qtype': 'susceptibility',
 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?',
 'Answer': 'LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'}

In [7]:
from transformers import T5ForConditionalGeneration
#get a pre-trained model
model_id="google/flan-t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="finetune-t5-demo",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
   # push_to_hub=True,
    remove_unused_columns=False
)

In [11]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    prefix = "answer the question: "
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["Question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True,padding='max_length')

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["Answer"], max_length=512, truncation=True,padding='max_length')
    #model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [13]:
tokenized_medqa = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [14]:
tokenized_medqa


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16407
    })
})

In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [16]:
tokenized_medqa_train = tokenized_medqa["train"].shuffle(seed=42).select(range(1000))
tokenized_medqa_test = tokenized_medqa["train"].shuffle(seed=42).select(range(1000))
tokenized_medqa_train

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

In [71]:
tokenized_medqa_train.data

MemoryMappedTable
input_ids: list<item: int32>
  child 0, item: int32
attention_mask: list<item: int8>
  child 0, item: int8
----
input_ids: [[[1525,8,822,10,2645,...,0,0,0,0,0],[1525,8,822,10,363,...,0,0,0,0,0],...,[1525,8,822,10,363,...,0,0,0,0,0],[1525,8,822,10,363,...,0,0,0,0,0]],[[1525,8,822,10,363,...,0,0,0,0,0],[1525,8,822,10,125,...,0,0,0,0,0],...,[1525,8,822,10,363,...,0,0,0,0,0],[1525,8,822,10,2645,...,0,0,0,0,0]],...,[[1525,8,822,10,571,...,0,0,0,0,0],[1525,8,822,10,363,...,0,0,0,0,0],...,[1525,8,822,10,363,...,0,0,0,0,0],[1525,8,822,10,27,...,0,0,0,0,0]],[[1525,8,822,10,571,...,0,0,0,0,0],[1525,8,822,10,363,...,0,0,0,0,0],...,[1525,8,822,10,363,...,0,0,0,0,0],[1525,8,822,10,363,...,0,0,0,0,0]]]
attention_mask: [[[1,1,1,1,1,...,0,0,0,0,0],[1,1,1,1,1,...,0,0,0,0,0],...,[1,1,1,1,1,...,0,0,0,0,0],[1,1,1,1,1,...,0,0,0,0,0]],[[1,1,1,1,1,...,0,0,0,0,0],[1,1,1,1,1,...,0,0,0,0,0],...,[1,1,1,1,1,...,0,0,0,0,0],[1,1,1,1,1,...,0,0,0,0,0]],...,[[1,1,1,1,1,...,0,0,0,0,0],[1,1,1,1,1,...,0

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_medqa_train,
    eval_dataset=tokenized_medqa_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.000449
2,No log,0.000097
3,No log,0.000058


TrainOutput(global_step=375, training_loss=0.061268157958984375, metrics={'train_runtime': 13758.9601, 'train_samples_per_second': 0.218, 'train_steps_per_second': 0.027, 'total_flos': 513568014336000.0, 'train_loss': 0.061268157958984375, 'epoch': 3.0})

In [19]:
trainer.save_model("test-t5-med-trained")

In [30]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
trainer.push_to_hub('jayeshvpatil/finetune-t5-medqa')

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1706788637.17f464fda1d5.122.0:   0%|          | 0.00/6.26k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jayeshvpatil/finetune-t5-demo/commit/61c60157747a5c7a5694768a3bc38562a4ffa802', commit_message='jayeshvpatil/finetune-t5-medqa', commit_description='', oid='61c60157747a5c7a5694768a3bc38562a4ffa802', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
from transformers import pipeline
question = "What is yellow fever vaccination?"
question_answerer = pipeline("question-answering", model="jayeshvpatil/finetune-t5-demo")
question_answerer(Question=question)

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at jayeshvpatil/finetune-t5-demo and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: Unknown arguments {'Question': 'What is yellow fever vaccination?'}

In [72]:
from  transformers  import  AutoTokenizer, T5ForConditionalGeneration, pipeline

model_name = "jayeshvpatil/finetune-t5-demo"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
question = "Who is at risk for Lymphocytic Choriomeningitis (LCM)?"
input = f" answer :{question}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


answer : Who is at risk for Lymphocytic Choriomeningit


In [73]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("jayeshvpatil/finetune-t5-demo")
model = T5ForConditionalGeneration.from_pretrained("jayeshvpatil/finetune-t5-demo")

encoded_input = tokenizer("Who is at risk for Lymphocytic Choriomeningitis (LCM)?", return_tensors="pt")
outputs = model.generate(input_ids=encoded_input.input_ids,attention_mask = encoded_input.attention_mask)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


a person with a history of a bacterial infection
